# SmartSwitch Tutorial 03: Value-Based Dispatch

**Welcome back!** This is the third tutorial in the SmartSwitch series.

In this notebook you'll learn:
- ✅ How to dispatch automatically based on runtime values
- ✅ How to eliminate long if/elif chains
- ✅ How to build declarative routing logic

**Time**: ~10 minutes

**Prerequisites**: Complete Tutorials 01-02 first

---

## The Problem

You need to choose which handler to call based on the **values** of arguments.

**Traditional approach** - Long if/elif chains:

In [ ]:
# Traditional: Imperative if/elif (messy!)
def handle_request(method, path, data=None):
    if method == 'GET' and path == '/users':
        return list_users()
    elif method == 'POST' and path == '/users':
        return create_user(data)
    elif method == 'GET' and path.startswith('/users/'):
        user_id = path.split('/')[-1]
        return get_user(user_id)
    elif method == 'DELETE' and path.startswith('/users/'):
        user_id = path.split('/')[-1]
        return delete_user(user_id)
    else:
        return {'error': 'Not found'}

def list_users():
    return ["Alice", "Bob"]

def create_user(data):
    return f"Created user: {data}"

def get_user(user_id):
    return f"User {user_id}"

def delete_user(user_id):
    return f"Deleted user {user_id}"

# Test
print(handle_request('GET', '/users'))
print(handle_request('POST', '/users', {'name': 'Charlie'}))

**Problems:**
- ❌ Routing logic mixed with handler definitions
- ❌ Hard to test individual routes
- ❌ Difficult to add new routes
- ❌ One giant function that keeps growing

## The SmartSwitch Solution

Use `valrule` to declare routing conditions:

In [ ]:
from smartswitch import Switcher

api = Switcher()

# Each handler declares its own routing rule!
@api(valrule=lambda method, path: method == 'GET' and path == '/users')
def list_users(method, path, data=None):
    return ["Alice", "Bob"]

@api(valrule=lambda method, path: method == 'POST' and path == '/users')
def create_user(method, path, data=None):
    return f"Created user: {data}"

# Default handler (no valrule = matches anything)
@api
def not_found(method, path, data=None):
    return {"error": "Not found"}

# Automatic dispatch!
print(api()('GET', '/users'))
print(api()('POST', '/users', {'name': 'Charlie'}))
print(api()('PUT', '/users'))  # Falls through to not_found

**How it works:**

```python
api()  # ← Returns dispatcher (no arguments!)
      # ↓ Call with actual arguments
api()('GET', '/users')
```

SmartSwitch:
1. Tests each `valrule` with the arguments
2. Calls the **first matching handler**
3. Falls through to default handler if no match

## Benefits

✅ **Declarative** - Routing rules near their handlers

✅ **Testable** - Each handler is independent

✅ **Extensible** - Just add another `@api(valrule=...)`

✅ **No central router** - No giant if/elif to maintain

## Valrule Patterns

### Pattern 1: Single Parameter Check

In [ ]:
from smartswitch import Switcher

processor = Switcher()

@processor(valrule=lambda status: status == 'pending')
def handle_pending(status, data):
    return f"Processing pending: {data}"

@processor(valrule=lambda status: status == 'completed')
def handle_completed(status, data):
    return f"Archiving completed: {data}"

@processor
def handle_other(status, data):
    return f"Unknown status: {status}"

print(processor()('pending', 'task_123'))
print(processor()('completed', 'task_456'))
print(processor()('failed', 'task_789'))

### Pattern 2: Multiple Parameter Check

In [ ]:
payments = Switcher()

# Match when BOTH conditions are true
@payments(valrule=lambda method, amount: method == 'crypto' and amount > 1000)
def process_large_crypto(method, amount, details):
    return {"processor": "crypto_large", "fee": amount * 0.01}

@payments(valrule=lambda method, **kw: method == 'credit_card')
def process_card(method, amount, details):
    return {"processor": "credit_card", "fee": amount * 0.03}

@payments
def process_generic(method, amount, details):
    return {"error": "Unsupported payment method"}

print(payments()('crypto', 5000, {}))
print(payments()('credit_card', 100, {}))

### Pattern 3: Dict-Style Access

When you have many parameters, use `**kw`:

In [ ]:
users = Switcher()

# Access via dict
@users(valrule=lambda kw: kw['user_type'] == 'admin' and kw['action'] == 'delete')
def admin_delete(user_type, action, target):
    return f"Admin deleting: {target}"

@users(valrule=lambda kw: kw['user_type'] == 'guest')
def guest_action(user_type, action, target):
    return "Guests cannot perform actions"

@users
def regular_user(user_type, action, target):
    return f"Regular user {action} on {target}"

print(users()(user_type='admin', action='delete', target='post_123'))
print(users()(user_type='guest', action='edit', target='post_456'))

## Try It Yourself!

Build a state machine for order processing:

In [ ]:
from smartswitch import Switcher

order_fsm = Switcher()

@order_fsm(valrule=lambda state, event: state == 'pending' and event == 'pay')
def transition_to_paid(state, event, order_id):
    return {'new_state': 'paid', 'order_id': order_id}

# TODO: Add more transitions!
# - paid + ship -> shipped
# - shipped + deliver -> delivered
# - any + cancel -> cancelled

print(order_fsm()('pending', 'pay', 'ORDER_123'))

## Real-World Example: HTTP API Router

A complete REST API with method + path routing:

In [ ]:
from smartswitch import Switcher

http_api = Switcher()

# Users endpoints
@http_api(valrule=lambda method, path: method == 'GET' and path == '/users')
def list_users(method, path, body=None):
    return {"users": ["alice", "bob"]}

@http_api(valrule=lambda method, path: method == 'POST' and path == '/users')
def create_user(method, path, body=None):
    return {"created": body.get("name")}

@http_api(valrule=lambda method, path: method == 'GET' and path.startswith('/users/'))
def get_user(method, path, body=None):
    user_id = path.split('/')[-1]
    return {"user_id": user_id, "name": "Alice"}

# Posts endpoints
@http_api(valrule=lambda method, path: method == 'GET' and path == '/posts')
def list_posts(method, path, body=None):
    return {"posts": ["post1", "post2"]}

# Default handler
@http_api
def not_found(method, path, body=None):
    return {"error": "Not Found", "status": 404}

# Simulate HTTP requests
print("GET /users:", http_api()('GET', '/users'))
print("POST /users:", http_api()('POST', '/users', {"name": "Charlie"}))
print("GET /users/123:", http_api()('GET', '/users/123'))
print("GET /posts:", http_api()('GET', '/posts'))
print("GET /unknown:", http_api()('GET', '/unknown'))

## Complex Conditions

Use regular Python in your valrules:

In [ ]:
import re
from smartswitch import Switcher

router = Switcher()

# Regex matching
@router(valrule=lambda path: re.match(r'^/api/v\d+/users$', path))
def api_users(path, method='GET'):
    version = re.search(r'v(\d+)', path).group(1)
    return f"API v{version} users endpoint"

# Range check
@router(valrule=lambda priority: 1 <= priority <= 5)
def high_priority(priority, task):
    return f"High priority task: {task}"

# List membership
@router(valrule=lambda action: action in ['read', 'write', 'delete'])
def valid_action(action, resource):
    return f"Performing {action} on {resource}"

print(router()(path='/api/v2/users'))
print(router()(priority=3, task='urgent'))
print(router()(action='read', resource='file.txt'))

## Order Matters!

SmartSwitch uses **first match wins** - more specific rules should come first:

In [ ]:
priority_router = Switcher()

# ✅ CORRECT: More specific first
@priority_router(valrule=lambda status, priority: status == 'urgent' and priority > 5)
def handle_critical(status, priority):
    return "CRITICAL!"

@priority_router(valrule=lambda status, **kw: status == 'urgent')
def handle_urgent(status, priority):
    return "Urgent"

@priority_router
def handle_normal(status, priority):
    return "Normal"

print(priority_router()('urgent', 8))  # → CRITICAL!
print(priority_router()('urgent', 3))  # → Urgent
print(priority_router()('normal', 1))  # → Normal

## When to Use Value-Based Dispatch

This pattern is perfect for:

✅ **HTTP routing** - Method + path + headers

✅ **State machines** - State + event transitions

✅ **Business rules** - Complex conditional logic

✅ **Event handlers** - Event type + attributes

⚠️ **Consider alternatives for**:
- Simple 2-3 case switches → use `if/elif`
- Type-based dispatch → See Tutorial 04

## Exercise: Build an Event Bus

Create an event bus that routes events by type and priority:

In [ ]:
from smartswitch import Switcher

event_bus = Switcher()

@event_bus(valrule=lambda event_type, priority: event_type == 'error' and priority == 'critical')
def handle_critical_error(event_type, priority, message):
    return f"🚨 ALERT: {message}"

# TODO: Add more event handlers!
# - warning events
# - info events
# - debug events

print(event_bus()(event_type='error', priority='critical', message='System down!'))

## Summary

You learned:

✅ **Declarative routing** with `valrule=lambda ...`

✅ **Automatic dispatch** with `switcher()(args)`

✅ **First match wins** - Order rules carefully

✅ **Complex conditions** - Use full Python expressions

---

## Next Steps

Continue to **Tutorial 04: Type-Based Dispatch** to learn how to route based on argument **types** instead of values.

📖 **Documentation**: [Value Rules Guide](https://smartswitch.readthedocs.io/guide/valrules/)

---

**Questions?** Open an issue on [GitHub](https://github.com/genropy/smartswitch/issues)